In [18]:
from datetime import time
from decimal import Decimal

import pandas as pd

from nautilus_trader.adapters.databento.loaders import DatabentoDataLoader
from nautilus_trader.backtest.engine import BacktestEngine, BacktestEngineConfig
from nautilus_trader.common.enums import LogColor
from nautilus_trader.config import ExecAlgorithmConfig, LoggingConfig, PositiveInt, StrategyConfig
from nautilus_trader.core.correctness import PyCondition
from nautilus_trader.core.data import Data
from nautilus_trader.core.datetime import dt_to_unix_nanos, unix_nanos_to_dt
from nautilus_trader.core.message import Event
from nautilus_trader.execution.algorithm import ExecAlgorithm
from nautilus_trader.indicators.average.ema import ExponentialMovingAverage
from nautilus_trader.model.book import OrderBook
from nautilus_trader.model.currencies import USD
from nautilus_trader.model.data import Bar, BarType, OrderBookDeltas, QuoteTick, TradeTick
from nautilus_trader.model.enums import AccountType, BarAggregation, OmsType, OrderSide, TimeInForce
from nautilus_trader.model.identifiers import ExecAlgorithmId, InstrumentId, TraderId, Venue
from nautilus_trader.model.instruments import Instrument
from nautilus_trader.model.objects import Money
from nautilus_trader.model.orders import MarketOrder, Order, OrderList
from nautilus_trader.persistence.loaders import CSVBarDataLoader
from nautilus_trader.persistence.wranglers import BarDataWrangler
from nautilus_trader.test_kit.providers import TestInstrumentProvider
from nautilus_trader.trading.strategy import Strategy
from nautilus_trader.examples.strategies.ema_cross_long_only import EMACrossLongOnly, EMACrossLongOnlyConfig


In [21]:

class RangeBreakoutConfig(StrategyConfig):
    instrument_id: InstrumentId
    bar_type: BarType
    trade_size: Decimal
    range_start: time = time(16, 0)  # 16:00:00
    range_end: time = time(19, 0)    # 19:00:00
    exit_time: time = time(23, 59)   # 23:59:00
    profit_factor: float = 0.6      # Target multiplier
    loss_factor: float = 0.6        # Stop loss multiplier
    


class RangeBreakoutStrategy(Strategy):
    """
    A breakout strategy that trades based on price action relative to a time-based range.
    
    The strategy:
    1. Calculates high/low range between t1 (16:00) and t2 (19:00)
    2. After t2, enters long positions when price breaks above the range high
    3. Sets profit target and stop loss based on range size
    4. Exits any open position at t3 (23:59)
    """

    def __init__(self, config: RangeBreakoutConfig):
        super().__init__(config)
        
        # Configuration
        self.instrument_id = config.instrument_id
        self.bar_type = config.bar_type
        self.trade_size = config.trade_size
        self.range_start = config.range_start
        self.range_end = config.range_end
        self.exit_time = config.exit_time
        self.profit_factor = config.profit_factor
        self.loss_factor = config.loss_factor
        
        # Runtime variables
        self.instrument: Instrument = None
        self.range_high: float = None
        self.range_low: float = None
        self.in_range_building_time: bool = False
        self.range_built: bool = False
        self.position_opened: bool = False

    def on_start(self) -> None:
        """Actions to be performed on strategy start."""
        # Get instrument
        self.instrument = self.cache.instrument(self.instrument_id)
        if self.instrument is None:
            self.log.error(f"Could not find instrument for {self.instrument_id}")
            self.stop()
            return

        # Subscribe to bar data
        self.subscribe_bars(self.bar_type)

    def on_bar(self, bar: Bar) -> None:
        """
        Process incoming bars and implement trading logic.
        """
        # Skip bars with no price information
        if bar.is_single_price():
            return

        # bar_time = bar.timestamp.time()
        current_dt = unix_nanos_to_dt(bar.ts_init)
        bar_time = current_dt.time()
        
        # Check if we're in range building period
        if self.range_start <= bar_time < self.range_end:
            self.in_range_building_time = True
            
            # Update range high/low
            if self.range_high is None or bar.high.as_double() > self.range_high:
                self.range_high = bar.high.as_double()
            if self.range_low is None or bar.low.as_double() < self.range_low:
                self.range_low = bar.low.as_double()
                
        # Range building period is over
        elif bar_time >= self.range_end and self.in_range_building_time:
            self.in_range_building_time = False
            self.range_built = True
            self.log.info(
                f"Range built - High: {self.range_high}, Low: {self.range_low}",
                color=LogColor.CYAN,
            )
            
        # Trading period
        if self.range_built and not self.position_opened and bar_time >= self.range_end:
            # Check for breakout
            if bar.close.as_double() > self.range_high:
                self.enter_long_position(bar.close.as_double())
                
        # Check for exit time
        if bar_time >= self.exit_time:
            if self.portfolio.is_net_long(self.instrument_id):
                self.close_all_positions(self.instrument_id)
                self.log.info("Time exit executed", color=LogColor.YELLOW)
            
            # Reset for next day
            self.reset_daily_vars()

    def enter_long_position(self, entry_price: float) -> None:
        """Enter a long position and store target/stop prices for order handling."""
        range_size = self.range_high - self.range_low
        
        # Store prices for order handling
        self.pending_entry_price = entry_price
        self.pending_target_price = entry_price + (range_size * self.profit_factor)
        self.pending_stop_price = entry_price - (range_size * self.loss_factor)
        
        # Create and submit market order
        order = self.order_factory.market(
            instrument_id=self.instrument_id,
            order_side=OrderSide.BUY,
            quantity=self.instrument.make_qty(self.trade_size),
            time_in_force=TimeInForce.IOC,
        )

        self.submit_order(order)

    def on_order(self, order: Order) -> None:
        """
        Handles order status updates and places take profit/stop loss orders
        when entry is filled.
        """
        if order.status == OrderStatus.FILLED:
            if isinstance(order, MarketOrder) and order.side == OrderSide.BUY:
                # Entry order was filled, place take profit and stop loss
                if self.pending_target_price and self.pending_stop_price:
                    # Place take profit order
                    tp_order = self.order_factory.limit(
                        instrument_id=self.instrument_id,
                        order_side=OrderSide.SELL,
                        price=Price(self.pending_target_price),
                        quantity=order.quantity,
                        time_in_force=TimeInForce.GTC,
                    )
                    self.submit_order(tp_order)

                    # Place stop loss order
                    sl_order = self.order_factory.stop_market(
                        instrument_id=self.instrument_id,
                        order_side=OrderSide.SELL,
                        trigger_price=Price(self.pending_stop_price),
                        quantity=order.quantity,
                        time_in_force=TimeInForce.GTC,
                    )
                    self.submit_order(sl_order)

                    self.position_opened = True
                    self.log.info(
                        f"Long position opened - Entry: {self.pending_entry_price}, "
                        f"Target: {self.pending_target_price}, Stop: {self.pending_stop_price}",
                        color=LogColor.GREEN,
                    )

                    # Clear pending prices
                    self.pending_entry_price = None
                    self.pending_target_price = None
                    self.pending_stop_price = None

    def reset_daily_vars(self) -> None:
        """Reset variables for the next trading day."""
        self.range_high = None
        self.range_low = None
        self.in_range_building_time = False
        self.range_built = False
        self.position_opened = False
        self.pending_entry_price = None
        self.pending_target_price = None
        self.pending_stop_price = None
        

    def on_stop(self) -> None:
        """Actions to be performed when the strategy is stopped."""
        self.cancel_all_orders(self.instrument_id)
        self.close_all_positions(self.instrument_id)
        self.unsubscribe_bars(self.bar_type)

    def on_reset(self) -> None:
        """Actions to be performed when the strategy is reset."""
        self.reset_daily_vars()

    def on_save(self) -> dict[str, bytes]:
        """Save strategy state."""
        return {}

    def on_load(self, state: dict[str, bytes]) -> None:
        """Load strategy state."""
        pass

    def on_dispose(self) -> None:
        """Clean up resources."""
        pass

In [23]:

if __name__ == "__main__":
    config = BacktestEngineConfig(
        trader_id=TraderId("BACKTESTER-001"),
        logging=LoggingConfig(log_level="INFO"),
    )
    engine = BacktestEngine(config=config)

    NASDAQ = Venue("XNAS")

    engine.add_venue(
        venue=NASDAQ,
        oms_type=OmsType.NETTING,
        account_type=AccountType.CASH,
        base_currency=USD,
        starting_balances=[Money(1_000_000_000.0, USD)],
    )

    SPY_XNAS = TestInstrumentProvider.equity(symbol="NQ", venue="XNAS")
    engine.add_instrument(SPY_XNAS)
    instrument_id = SPY_XNAS.id

    bar_type = BarType.from_str(f"{instrument_id}-1-MINUTE-LAST-EXTERNAL")
    wrangler = BarDataWrangler(bar_type, SPY_XNAS)

    df = pd.read_csv("./Nasdaq_1min_v1.csv", index_col=0)
    df1 = df.copy()
    df1['timestamp'] = pd.to_datetime(df1['ts_event'])
    df1.set_index('timestamp', inplace=True)
    df1 = df1.iloc[:30000]

    bars = wrangler.process(
        df1[['open', 'high', 'low', 'close', 'volume']],
        ts_init_delta=60000000000
    )

    engine.add_data(bars)

    config = RangeBreakoutConfig(
        instrument_id=SPY_XNAS.id,
        bar_type=BarType.from_str(f"{SPY_XNAS.id}-1-MINUTE-LAST-EXTERNAL"), #BarAggregation.MINUTE,
        trade_size=Decimal("1"),
    )

# RangeBreakoutStrategy
# RangeBreakoutConfig
    # strategy = EMACrossLongOnly(config=config)
    # engine.add_strategy(strategy=strategy)

    strategy = RangeBreakoutStrategy(config=config)
    engine.add_strategy(strategy)

    engine.run()

In [24]:
account_report_df = engine.trader.generate_account_report(NASDAQ)
account_report_df

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2019-08-21 00:01:00+00:00,1000000000.00,0.00,1000000000.00,USD,XNAS-001,CASH,USD,[],True,{}
2019-08-21 20:37:00+00:00,999992242.25,0.00,999992242.25,USD,XNAS-001,CASH,USD,[],False,{}
2019-08-21 20:38:00+00:00,999984484.75,0.00,999984484.75,USD,XNAS-001,CASH,USD,[],False,{}
2019-08-21 20:52:00+00:00,999976727.75,0.00,999976727.75,USD,XNAS-001,CASH,USD,[],False,{}
2019-08-21 20:53:00+00:00,999968970.25,0.00,999968970.25,USD,XNAS-001,CASH,USD,[],False,{}
...,...,...,...,...,...,...,...,...,...,...
2019-09-18 23:58:00+00:00,998798783.23,0.00,998798783.23,USD,XNAS-001,CASH,USD,[],False,{}
2019-09-18 23:59:00+00:00,998790897.48,0.00,998790897.48,USD,XNAS-001,CASH,USD,[],False,{}
2019-09-18 23:59:00+00:00,998798783.23,0.00,998798783.23,USD,XNAS-001,CASH,USD,[],False,{}
2019-09-18 23:59:00+00:00,999989529.97,0.00,999989529.97,USD,XNAS-001,CASH,USD,[],False,{}


In [25]:
order_fills_report_df = engine.trader.generate_order_fills_report()
order_fills_report_df

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,init_id,ts_init,ts_last
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20190821-203700-001-000-1,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-1-001,NQ.XNAS-RangeBreakoutStrategy-000,XNAS-001,XNAS-1-1223,MARKET,BUY,1,...,None,None,None,None,None,None,None,1f18d3ad-87fb-4cd4-8654-4a3e6076f4fe,2019-08-21 20:37:00+00:00,2019-08-21 20:37:00+00:00
O-20190821-203800-001-000-2,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-1-002,NQ.XNAS-RangeBreakoutStrategy-000,XNAS-001,XNAS-1-1225,MARKET,BUY,1,...,None,None,None,None,None,None,None,c4dd8170-05b8-46ab-800f-fe872673e568,2019-08-21 20:38:00+00:00,2019-08-21 20:38:00+00:00
O-20190821-205200-001-000-3,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-1-003,NQ.XNAS-RangeBreakoutStrategy-000,XNAS-001,XNAS-1-1240,MARKET,BUY,1,...,None,None,None,None,None,None,None,d752d0f4-5c10-4700-ad89-5a95df5301cd,2019-08-21 20:52:00+00:00,2019-08-21 20:52:00+00:00
O-20190821-205300-001-000-4,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-1-004,NQ.XNAS-RangeBreakoutStrategy-000,XNAS-001,XNAS-1-1242,MARKET,BUY,1,...,None,None,None,None,None,None,None,580a2c5a-c99f-47f9-a475-e3208f5f4af3,2019-08-21 20:53:00+00:00,2019-08-21 20:53:00+00:00
O-20190821-205400-001-000-5,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-1-005,NQ.XNAS-RangeBreakoutStrategy-000,XNAS-001,XNAS-1-1244,MARKET,BUY,1,...,None,None,None,None,None,None,None,32d47751-0e1a-4942-980a-a2ef3d2b1970,2019-08-21 20:54:00+00:00,2019-08-21 20:54:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O-20190918-235700-001-000-1146,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-1-1146,NQ.XNAS-RangeBreakoutStrategy-000,XNAS-001,XNAS-1-29588,MARKET,BUY,1,...,None,None,None,None,None,None,None,1f2825a2-62c3-4f3b-b90a-53d75f7c4167,2019-09-18 23:57:00+00:00,2019-09-18 23:57:00+00:00
O-20190918-235800-001-000-1147,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-1-1147,NQ.XNAS-RangeBreakoutStrategy-000,XNAS-001,XNAS-1-29590,MARKET,BUY,1,...,None,None,None,None,None,None,None,cc3dffe6-d22f-44ba-91bb-9eb586733e70,2019-09-18 23:58:00+00:00,2019-09-18 23:58:00+00:00
O-20190918-235900-001-000-1148,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-1-1148,NQ.XNAS-RangeBreakoutStrategy-000,XNAS-001,XNAS-1-29592,MARKET,BUY,1,...,None,None,None,None,None,None,None,2a4ae3a0-a0cd-4a94-ad59-c3565b9474c2,2019-09-18 23:59:00+00:00,2019-09-18 23:59:00+00:00


In [29]:
positions_report_df = engine.trader.generate_positions_report()
positions_report_df

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,,
NQ.XNAS-RangeBreakoutStrategy-000-0c082c39-6e06-46d2-8ba1-fd2d8052d171,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190821-203700-001-000-1,O-20190822-235900-001-000-130,BUY,FLAT,0,124,2019-08-21 20:37:00+00:00,1566518340000000000,2019-08-22 23:59:00+00:00,98520000000000,7746.185483870968,7768.6675,['0.00 USD'],0.0029,-165.57 USD
NQ.XNAS-RangeBreakoutStrategy-000-aeb1bc96-23ff-4582-9a80-eadf41ec7e6a,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190826-200000-001-000-131,O-20190826-235900-001-000-254,BUY,FLAT,0,123,2019-08-26 20:00:00+00:00,1566863940000000000,2019-08-26 23:59:00+00:00,14340000000000,7589.477642276423,7572.741138211382,['0.00 USD'],-0.00221,-2058.59 USD
NQ.XNAS-RangeBreakoutStrategy-000-09ea2547-b8d6-4f6a-b5a4-d69204b5e0fe,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190827-190300-001-000-255,O-20190827-235900-001-000-263,BUY,FLAT,0,8,2019-08-27 19:03:00+00:00,1566950340000000000,2019-08-27 23:59:00+00:00,17760000000000,7599.15625,7587.5,['0.00 USD'],-0.00153,-93.25 USD
NQ.XNAS-RangeBreakoutStrategy-000-aa0184a8-e8f6-4c21-8b83-548830de5275,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190828-192000-001-000-264,O-20190828-235900-001-000-267,BUY,FLAT,0,3,2019-08-28 19:20:00+00:00,1567036740000000000,2019-08-28 23:59:00+00:00,16740000000000,7601.583333333333,7580.0,['0.00 USD'],-0.00284,-64.75 USD
NQ.XNAS-RangeBreakoutStrategy-000-bb1bc3a9-b184-4f0d-a1c8-d10ef0ad0894,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190902-225500-001-000-268,O-20190902-235900-001-000-273,BUY,FLAT,0,5,2019-09-02 22:55:00+00:00,1567468740000000000,2019-09-02 23:59:00+00:00,3840000000000,7641.5,7627.25,['0.00 USD'],-0.00186,-71.25 USD
NQ.XNAS-RangeBreakoutStrategy-000-d66c16d3-9d63-47e9-a4a4-f4e4fe18f4fa,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190904-192500-001-000-274,O-20190908-235900-001-000-555,BUY,FLAT,0,183,2019-09-04 19:25:00+00:00,1567987140000000000,2019-09-08 23:59:00+00:00,362040000000000,7873.32865864071,7774.926523297491,['0.00 USD'],-0.0125,265.50 USD
NQ.XNAS-RangeBreakoutStrategy-000-d0b1ccf1-4600-496d-8c82-a6106564d3ef,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190910-200100-001-000-556,O-20190910-235900-001-000-627,BUY,FLAT,0,71,2019-09-10 20:01:00+00:00,1568159940000000000,2019-09-10 23:59:00+00:00,14280000000000,7815.68661971831,7811.741126760563,['0.00 USD'],-0.0005,-280.13 USD
NQ.XNAS-RangeBreakoutStrategy-000-a826f29f-1c83-4286-a606-8603a4afce1c,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190911-203800-001-000-628,O-20190912-235900-001-000-693,BUY,FLAT,0,64,2019-09-11 20:38:00+00:00,1568332740000000000,2019-09-12 23:59:00+00:00,98460000000000,7923.5,7953.13625,['0.00 USD'],0.00374,1896.72 USD
NQ.XNAS-RangeBreakoutStrategy-000,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190916-200300-001-000-694,O-20190919-235900-001-000-1150,BUY,FLAT,0,160,2019-09-16 20:03:00+00:00,1568937540000000000,2019-09-19 23:59:00+00:00,273360000000000,7895.225422664665,7880.182737306844,['0.00 USD'],-0.00191,-1971.48 USD


In [31]:
account_report_df = engine.trader.generate_account_report(NASDAQ)
order_fills_report_df = engine.trader.generate_order_fills_report()
positions_report_df = engine.trader.generate_positions_report()
positions_report_df

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,,
NQ.XNAS-RangeBreakoutStrategy-000-0c082c39-6e06-46d2-8ba1-fd2d8052d171,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190821-203700-001-000-1,O-20190822-235900-001-000-130,BUY,FLAT,0,124,2019-08-21 20:37:00+00:00,1566518340000000000,2019-08-22 23:59:00+00:00,98520000000000,7746.185483870968,7768.6675,['0.00 USD'],0.0029,-165.57 USD
NQ.XNAS-RangeBreakoutStrategy-000-aeb1bc96-23ff-4582-9a80-eadf41ec7e6a,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190826-200000-001-000-131,O-20190826-235900-001-000-254,BUY,FLAT,0,123,2019-08-26 20:00:00+00:00,1566863940000000000,2019-08-26 23:59:00+00:00,14340000000000,7589.477642276423,7572.741138211382,['0.00 USD'],-0.00221,-2058.59 USD
NQ.XNAS-RangeBreakoutStrategy-000-09ea2547-b8d6-4f6a-b5a4-d69204b5e0fe,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190827-190300-001-000-255,O-20190827-235900-001-000-263,BUY,FLAT,0,8,2019-08-27 19:03:00+00:00,1566950340000000000,2019-08-27 23:59:00+00:00,17760000000000,7599.15625,7587.5,['0.00 USD'],-0.00153,-93.25 USD
NQ.XNAS-RangeBreakoutStrategy-000-aa0184a8-e8f6-4c21-8b83-548830de5275,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190828-192000-001-000-264,O-20190828-235900-001-000-267,BUY,FLAT,0,3,2019-08-28 19:20:00+00:00,1567036740000000000,2019-08-28 23:59:00+00:00,16740000000000,7601.583333333333,7580.0,['0.00 USD'],-0.00284,-64.75 USD
NQ.XNAS-RangeBreakoutStrategy-000-bb1bc3a9-b184-4f0d-a1c8-d10ef0ad0894,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190902-225500-001-000-268,O-20190902-235900-001-000-273,BUY,FLAT,0,5,2019-09-02 22:55:00+00:00,1567468740000000000,2019-09-02 23:59:00+00:00,3840000000000,7641.5,7627.25,['0.00 USD'],-0.00186,-71.25 USD
NQ.XNAS-RangeBreakoutStrategy-000-d66c16d3-9d63-47e9-a4a4-f4e4fe18f4fa,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190904-192500-001-000-274,O-20190908-235900-001-000-555,BUY,FLAT,0,183,2019-09-04 19:25:00+00:00,1567987140000000000,2019-09-08 23:59:00+00:00,362040000000000,7873.32865864071,7774.926523297491,['0.00 USD'],-0.0125,265.50 USD
NQ.XNAS-RangeBreakoutStrategy-000-d0b1ccf1-4600-496d-8c82-a6106564d3ef,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190910-200100-001-000-556,O-20190910-235900-001-000-627,BUY,FLAT,0,71,2019-09-10 20:01:00+00:00,1568159940000000000,2019-09-10 23:59:00+00:00,14280000000000,7815.68661971831,7811.741126760563,['0.00 USD'],-0.0005,-280.13 USD
NQ.XNAS-RangeBreakoutStrategy-000-a826f29f-1c83-4286-a606-8603a4afce1c,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190911-203800-001-000-628,O-20190912-235900-001-000-693,BUY,FLAT,0,64,2019-09-11 20:38:00+00:00,1568332740000000000,2019-09-12 23:59:00+00:00,98460000000000,7923.5,7953.13625,['0.00 USD'],0.00374,1896.72 USD
NQ.XNAS-RangeBreakoutStrategy-000,BACKTESTER-001,RangeBreakoutStrategy-000,NQ.XNAS,XNAS-001,O-20190916-200300-001-000-694,O-20190919-235900-001-000-1150,BUY,FLAT,0,160,2019-09-16 20:03:00+00:00,1568937540000000000,2019-09-19 23:59:00+00:00,273360000000000,7895.225422664665,7880.182737306844,['0.00 USD'],-0.00191,-1971.48 USD


In [33]:
positions_report_df['avg_px_open'] = pd.to_numeric(positions_report_df['avg_px_open'], errors='coerce')
positions_report_df['avg_px_close'] = pd.to_numeric(positions_report_df['avg_px_close'], errors='coerce')
positions_report_df['realized_pnl'] = pd.to_numeric(positions_report_df['realized_pnl'].str.replace(' USD', ''), errors='coerce')

# Create new DataFrame with required columns
new_df = pd.DataFrame()
new_df['Entry price'] = positions_report_df['avg_px_open']
new_df['Exit price'] = positions_report_df['avg_px_close']
new_df['Net Profit'] = positions_report_df['realized_pnl']

# Calculate cumulative net profit
new_df['Cum Net Profit'] = new_df['Net Profit'].cumsum()

# Calculate drawdown as the difference from the running max Cum Net Profit
new_df['Drawdown'] = new_df['Cum Net Profit'] - new_df['Cum Net Profit'].cummax()

# Calculate net profit and max drawdown
net_profit = new_df['Net Profit'].sum()
max_drawdown = new_df['Drawdown'].min()

# Output the new DataFrame and calculated values
print(f"Net Profit: {net_profit}")
print(f"Max Drawdown: {max_drawdown}")
new_df

Net Profit: -2542.8
Max Drawdown: -2377.23


,Entry price,Exit price,Net Profit,Cum Net Profit,Drawdown
position_id,,,,,
NQ.XNAS-RangeBreakoutStrategy-000-0c082c39-6e06-46d2-8ba1-fd2d8052d171,7746.185484,7768.667500,-165.57,-165.57,0.00
NQ.XNAS-RangeBreakoutStrategy-000-aeb1bc96-23ff-4582-9a80-eadf41ec7e6a,7589.477642,7572.741138,-2058.59,-2224.16,-2058.59
NQ.XNAS-RangeBreakoutStrategy-000-09ea2547-b8d6-4f6a-b5a4-d69204b5e0fe,7599.156250,7587.500000,-93.25,-2317.41,-2151.84
NQ.XNAS-RangeBreakoutStrategy-000-aa0184a8-e8f6-4c21-8b83-548830de5275,7601.583333,7580.000000,-64.75,-2382.16,-2216.59
NQ.XNAS-RangeBreakoutStrategy-000-bb1bc3a9-b184-4f0d-a1c8-d10ef0ad0894,7641.500000,7627.250000,-71.25,-2453.41,-2287.84
NQ.XNAS-RangeBreakoutStrategy-000-d66c16d3-9d63-47e9-a4a4-f4e4fe18f4fa,7873.328659,7774.926523,265.50,-2187.91,-2022.34
NQ.XNAS-RangeBreakoutStrategy-000-d0b1ccf1-4600-496d-8c82-a6106564d3ef,7815.686620,7811.741127,-280.13,-2468.04,-2302.47
NQ.XNAS-RangeBreakoutStrategy-000-a826f29f-1c83-4286-a606-8603a4afce1c,7923.500000,7953.136250,1896.72,-571.32,-405.75
NQ.XNAS-RangeBreakoutStrategy-000,7895.225423,7880.182737,-1971.48,-2542.80,-2377.23
